<a href="https://colab.research.google.com/github/wanwanliang/Code_for_Research_Projects/blob/main/Disease_Detection/CNNs_WSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Prepare data

## install and load packages

In [ ]:
!pip install geopandas
!pip install rasterio

In [2]:
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sklearn
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import google.colab
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir("/content/drive/My Drive/UMN_Research/Data/wsr/image_200_bb45")

In [5]:
from keras.layers import Input, Conv2D, Activation, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D, Flatten, ZeroPadding2D, MaxPooling2D, Add
from keras.activations import relu, softmax
from keras.models import Model
from keras import regularizers, initializers

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Design Resnet

### Define identity block

In [7]:
def identity_block(X,f,filters, stage, block):

  conv_name_base = 'res' + str(stage) + block + "_branch"
  bn_name_base = 'bn' + str(stage) + block + "_branch"

  F1, F2, F3 = filters

  X_shortcut = X

  X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name = conv_name_base + '2a', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name=bn_name_base + '2b')(X)

  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

### Define convolutional block

In [8]:
def convolutional_block(X, f, filters, stage, block, s=2):

  conv_name_base = 'res' + str(stage) + block + "_branch"
  bn_name_base = 'bn' + str(stage) + block + "_branch"

  F1, F2, F3 = filters

  X_shortcut = X

  X = Conv2D(F1, (1,1), strides = (s,s), name= conv_name_base + "2a", kernel_initializer= initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters= F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer= initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=conv_name_base + '2c', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  ##### SHORTCUT PATH #### 
  X_shortcut =  Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = initializers.glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation 
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

### Create ResNet50

In [12]:
def ResNet50(input_shape = (30, 30, 5), classes=2):

  X_input = Input(input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  # stage 1 
  X = Conv2D(64, (3,3), strides=(2,2), name='conv1', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name='bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3), strides=(2,2), padding='same')(X)
  
  # stage 2
  X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
  X = identity_block(X, 3, [64,64,256], stage=2, block='b')
  X = identity_block(X, 3, [64,64,256], stage=2, block='c')

  # stage 3
  X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
  X = identity_block(X, 3, [128,128,512], stage=3, block='c')
  X = identity_block(X, 3, [128,128,512], stage=3, block='d')

  # stage 4
  X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

  # stage 5
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512,512, 2048], stage=5, block='b')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

  # averge pooling
  X = AveragePooling2D(name='avg_pool', padding='same')(X)

  # output layer
  X = Flatten()(X)
  X = Dense(2, activation='softmax', name='fc' + str(classes), kernel_initializer= initializers.glorot_uniform(seed=0))(X)

  # create model
  model = Model(inputs = X_input, outputs= X, name='ResNet50')

  return model

In [13]:
model = ResNet50(input_shape=(30, 30, 5), classes=2)

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

## Load and prepare data

In [15]:
os.chdir('/content/drive/Shared drives/WSR_data/Drone200ft/multispectral_reflectance/plot_images')

### list and order files

In [16]:
import glob
t = glob.glob("*.tif")
# t2 = sorted(t)
# os.remove('plot621 (1).tif')
print(len(t))
t[:10]

960


['plot598.tif',
 'plot482.tif',
 'plot624.tif',
 'plot520.tif',
 'plot574.tif',
 'plot560.tif',
 'plot636.tif',
 'plot501.tif',
 'plot587.tif',
 'plot541.tif']

In [17]:
nbs = []
[nbs.append(int((td.split('plot')[1]).split('.')[0])) for td in t]
nbs[:10]

[598, 482, 624, 520, 574, 560, 636, 501, 587, 541]

In [18]:
all = zip(nbs, t)
sorted_all = sorted(all)

In [19]:
t_sorted = [x for y, x in sorted_all]

In [20]:
t_sorted[:10]

['plot1.tif',
 'plot2.tif',
 'plot3.tif',
 'plot4.tif',
 'plot5.tif',
 'plot6.tif',
 'plot7.tif',
 'plot8.tif',
 'plot9.tif',
 'plot10.tif']

### load all images in one numpy array

In [21]:
def tif2ary(tif):

  raA = rio.open(tif)
  arys = raA.read()

  arys= arys.astype('float32')
  arys =np.moveaxis(arys, 0, -1)

  
  return(arys)

In [22]:
dt = None

for tif in t_sorted:

  ary = tif2ary(tif)
  ary = ary[:30, :30:,]
  ary = ary.reshape((1, 30,30,5))

  if dt is None:
    dt = ary
  else:
    dt = np.concatenate((dt, ary), axis=0)

In [23]:
dt.shape

(960, 30, 30, 5)

In [24]:
y = pd.read_csv("../../labels.csv")
print(y.shape)
y[:5]

(960, 8)


,plot_ID,binary_1,score,resistance,resistance_class_4,binary_2,block,variety
0,1,1,50.0,S,S,S,2,DH058
1,2,1,5.0,RMR,MR,R,2,Faller
2,3,1,25.0,MSS,S,S,2,DH121
3,4,1,40.0,MSS,S,S,2,DH80
4,5,1,25.0,MSS,S,S,2,ROB


In [25]:
y.binary_1.value_counts()

0    483
1    477
Name: binary_1, dtype: int64

In [26]:
y = y['binary_1']
y = np.asarray(y).reshape((-1,1))
y.shape

(960, 1)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(dt, y, random_state=16, shuffle=True, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=16, test_size=0.2)

print("Train size is: {}".format(x_train.shape[0]))
print("Test size is: {}".format(x_test.shape[0]))
print("Validation size is: {}".format(x_val.shape[0]))

Train size is: 652
Test size is: 144
Validation size is: 164


In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/50
10/21 [=============>................] - ETA: 41s - loss: 2.8011 - accuracy: 0.5094